# SPP LAOS: NLSQ → NUTS workflow (Rogers defaults)

Defaults: n_harmonics=39, step_size=8, num_mode=2, wrapped rate inference.

In [ ]:
import numpy as np
import jax.numpy as jnp
from rheojax.core.data import RheoData
from rheojax.transforms import SPPDecomposer
from rheojax.pipeline.workflows import SPPAmplitudeSweepPipeline

In [ ]:
# synthetic LAOS amplitude sweep
omega=1.5
gamma_levels=jnp.array([0.2,0.4,0.8])
t=jnp.linspace(0,2*jnp.pi/omega,400)
def make_ds(g):
    strain=g*jnp.sin(omega*t)
    stress=60.0*g**0.7*jnp.sin(omega*t)
    return RheoData(x=np.array(t), y=np.array(stress), domain="time", metadata={"omega": float(omega), "gamma_0": float(g), "strain": np.array(strain)})
datasets=[make_ds(float(g)) for g in gamma_levels]

In [ ]:
# Decompose one amplitude with SPPDecomposer (defaults already set to Rogers parity)
from rheojax.transforms import SPPDecomposer
decomposer=SPPDecomposer(omega=float(omega), gamma_0=float(gamma_levels[0]))
decomposer.transform(datasets[0])
print({k: decomposer.results_[k] for k in ["sigma_sy","sigma_dy","S_factor","T_factor"]})

In [ ]:
# Pipeline: NLSQ warm-start then Bayesian (small samples for demo)
pipeline=SPPAmplitudeSweepPipeline(omega=float(omega))
pipeline.run(datasets, gamma_0_values=list(map(float,gamma_levels)))
pipeline.fit_model(bayesian=False, yield_type="static")
model=pipeline.get_model()
print(model.parameters)
# Bayesian demo (tiny for speed)
bayes = model.fit_bayesian(gamma_levels, pipeline.get_yield_stresses()["sigma_sy"], test_mode="oscillation", num_warmup=50, num_samples=80)
print(bayes.summary.loc[["sigma_sy_scale","sigma_sy_exp"],"mean"])

## Notes
- Defaults: n_harmonics=39, step_size=8, num_mode=2, wrapped strain-rate when missing.
- Use `yield_type="dynamic"` to fit dynamic yields.
- Mark full Bayesian runs as `slow` in tests; increase samples for real analyses.
